### Install the required modules

In [ ]:
!pip install qdrant-haystack
!pip install farm-haystack[inference]

### Make qdrant up and running using the below command
`docker run -p 6333:6333 -v qdrant_storage qdrant/qdrant`

Note: Here we are storing the embeddings in the qdrant_storage directory so that we don't need to generate them every time.


In [ ]:
from qdrant_haystack import QdrantDocumentStore
from haystack import Document
from haystack.nodes import EmbeddingRetriever

In [ ]:
document_store = QdrantDocumentStore(
    "localhost",
    port = 6333,
    index="test_data",
    content_field = "content",
    name_field = "name",
    embedding_field = "vector",
    embedding_dim=384,
    recreate_index=True,
    hnsw_config={"m": 16, "ef_construct": 64},
    on_disk_payload = True
)

In [ ]:
from os import listdir
doc_dir = "wiki_gameofthrones_txt1"
files_to_index = [doc_dir + "/" + f for f in listdir(doc_dir)]

In [ ]:
docs = []
for file in files_to_index:
    with open(file, "r", encoding='utf-8') as f:
        text = " ".join(f.readlines()).strip()
        doc = Document(text=text, content=text, meta={"file_name": file})
        docs.append(doc)

In [ ]:
document_store.write_documents(docs)

In [ ]:
retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model="sentence-transformers/multi-qa-MiniLM-L6-cos-v1",
    model_format="sentence_transformers",
)

In [ ]:
document_store.update_embeddings(retriever)